# SQL-3. Соединение таблиц

## 2. Соединение таблиц по ключу
### Объединяем таблицы без операторов
Существует несколько способов соединения таблиц. Мы познакомимся со всеми основными операторами, которые используются для этих нужд, но начнём с простого метода объединения таблиц — без операторов.

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе from.


Каждая запись, которая есть в таблице teams, будет соединена с каждой записью в таблице matches.

__Это действие также называют декартовым произведением таблиц.__


*Исправим невозможность получения информации из полученной таблицы со значениями*
В таблице teams есть столбец api_id, а таблица matches содержит столбцы home_team_api_id и away_team_api_id — это ключи таблиц, по которым они соединяются.

__Ключ — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).__

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие:
```sql
where home_team_api_id = api_id
```

Полный код будет иметь вид:
```sql
SELECT *
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id
```

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на

```sql
where away_team_api_id = api_id
```

Итак, мы только что объединили таблицы по ключу. Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.


Ключи бывают двух основных типов:

1. __Primary__ — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.
2. __Foreign__ — внешний ключ — представляет собой ссылку на другую таблицу.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, _id, _rk, _cd, _pk (от primary_key), _fk (от foreign_key) и другие.


__Обратите внимание! В данном датасете ключ api_id таблицы teams может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (home) или гостевой (away) команде.__

С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.

```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id
```

## 3. Знакомимся с JOIN

В прошлом юните для соединения таблиц мы использовали условие в разделе where, чтобы показать принцип работы оператора JOIN.

__JOIN — это оператор SQL, который позволяет соединять таблицы по условию.__

При использовании where мы получили нижеследующий код:

```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id
```

Теперь запишем его с помощью join:

```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM    
    sql.teams
JOIN sql.matches on home_team_api_id = api_id
```

### Синтаксис

Оператор JOIN упрощает процесс соединения таблиц. Его синтаксис можно представить следующим образом:
```sql
SELECT
  столбец1,
  столбец2,
  .....
FROM
  таблица1
JOIN таблица2 ON условие

```

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью JOIN можно соединить и более двух таблиц:

```sql
SELECT
  столбец1,
  столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие
JOIN таблица3 ON условие

```

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

→ К примеру, столбец id есть и в таблице matches, и в таблице teams. Такой запрос не будет обработан.

Что же делать в таком случае? Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.

```sql
SELECT
    teams.id
FROM
    sql.teams
JOIN sql.matches ON home_team_api_id = api_id
```
Заменяем teams.id на matches.id и получаем идентефикаторы из другой таблицы.

Зачастую названия таблиц слишком длинные, так что использовать их неудобно.Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — алиасы (от англ. alias).

Синтаксис для указания алиаса такой же, как и для названия столбца.
```sql
SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1 AS короткое_название_1
JOIN таблица2 AS короткое_название_2 ON условие
```

Название записывается без пробелов и операторов. Если необходимо записать название, в котором используются пробелы ("table 1"), то алиас можно обернуть в кавычки.

Важно! Обращаться по такому алиасу придётся также с помощью кавычек:

```sql
SELECT
	"table 1".столбец1,
	"table 2".столбец2,
	...
FROM
	таблица1 AS "table 1"
	JOIN таблица2 AS "table 2" ON условие
```

___Использование таких алиасов считается плохой практикой как минимум по причине того, что обращаться с такими алиасами неудобно.__

Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве. Ключевое слово as, как и в названии столбца, можно опустить в большинстве СУБД.


Кажется, теперь мы можем соединить таблицы между собой и создать своё табло со счётом матчей, как на спортивных сайтах!

Давайте с помощью запроса SQL получим таблицу, содержащую:

название домашней команды;
количество забитых домашней командой голов;
количество забитых гостевой командой голов;
название гостевой команды.

## 4. Фильтрация и агрегатные функции

### Работа с объединенными таблицами

Принцип построения запроса и порядок операторов такой же, как и с обычной таблицей.

Вспомним его:
```sql
SELECT...
FROM...
WHERE...
GROUP BY...
ORDER BY...
LIMIT...
```

К соединённым таблицам применимы функции фильтрации данных. Например, можно вывести id матчей, в которых команда Arsenal была гостевой.

```sql
select
    m.id
from
    sql.teams as t
    join sql.matches as m on m.away_team_api_id = t.api_id
where long_name = 'Arsenal'
```
__Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.__

Например, результат запроса:

```sql
select
    m.id as id_1,
    m.season,
    t.id as id_2,
    t.long_name
from
    sql.teams as t
    join sql.matches as m on m.away_team_api_id = t.api_id
```

можно разделить на две разные части:
![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7281095f07ca452dfb5963c7ea912fe7/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-u2-md3_4_1.png)

Одна часть - таблица matches с алиасом m, вторая - teams с алиасом t, но после соединения они являются одной таблицей.

__Таким образом, если вы отфильтруете данные по одной части таблицы, то другая, соединённая, часть пропадёт вместе с ней.__


Задача:
Напишите запрос, который выведет полное название домашней команды (long_name), количество голов домашней команды (home_goal) и количество голов гостевой команды (away_goal) в матчах, где домашней командой были команды с коротким названием 'GEN'. Отсортируйте запрос по id матча в порядке возрастания.

Решение:
```sql
select
    t.long_name as long_name,
    m.home_team_goals as home_goal,
    m.away_team_goals as away_goal
from
    sql.teams as t
    join sql.matches as m on t.api_id = m.home_team_api_id
where t.short_name = 'GEN'
order by m.id asc
```

_Также мы можем отфильтровать записи сразу по двум таблицам._

Например, можно оставить только записи, в которых короткое название домашней команды GEN и матчи сезона 2008/2009.

```sql
SELECT *
FROM    
    sql.matches m
    JOIN sql.teams t on t.api_id = m.home_team_api_id
WHERE
    t.short_name = 'GEN'
    AND m.season = '2008/2009'
```

__АГРЕГАЦИЯ ДАННЫХ__

К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

Например, мы можем вывести сумму голов матча, забитых командами, агрегированную по гостевым командам (совокупное количество голов в матче, забитых обеими командами, суммированное в разрезе гостевых команд).

```sql
SELECT
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals
FROM
    sql.matches m
    JOIN sql.teams t ON m.away_team_api_id = t.api_id
GROUP BY t.id
```
 В данном запросе была использована группировка по столбцу id таблицы teams, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

Мы можем использовать оператор HAVING для фильтрации сгруппированных данных. Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

1. Для начала посчитаем общее количество голов в матчах по сезонам.

```sql
SELECT
    m.season,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals
FROM sql.matches m
GROUP BY m.season

```
2. Затем добавим таблицу с командами, группировку по командам и условие фильтрации.

```sql
SELECT
    m.season,
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
GROUP BY m.season, t.id
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100

```

## 5. Способы соединения таблиц
### Операторы
В прошлых юнитах для соединения таблиц мы использовали один оператор — JOIN. В действительности же он не один: существует несколько различных видов соединений (join’ов).

### INNER JOIN
Для INNER JOIN работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

В таблице teams есть данные о 299 различных командах — можем проверить это с помощью запроса.

```sql
select
    count(distinct id)
from
    sql.teams
```

Теперь добавим к teams таблицу с матчами.

```sql
select
    count(distinct t.id)
from
    sql.teams t
    join sql.matches m  on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
```

И в таблице останется уже не 299 команд, а только 292. Дело в том, что таблица sql.matches по какой-то причине не содержит информацию о командах Lierse SK, KVC Westerlo, KAS Eupen, Club Brugge KV, KV Oostende, RSC Anderlecht и Hull City, зато они есть в таблице sql.teams. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.


### LEFT OUTER JOIN И RIGHT OUTER JOIN

Также существуют схожие друг с другом типы соединения — LEFT JOIN и RIGHT JOIN (слово outer в операторе можно опустить).

Для LEFT JOIN работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.

Поставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице matches.

```sql
SELECT
    t.long_name
FROM
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE m.id IS NULL
```

Обратите внимание! Если мы добавим какой-либо фильтр по значению для таблицы matches, то LEFT JOIN превратится в INNER JOIN, поскольку для второй таблицы станет необходимым присутствие значения в строке.

```sql

SELECT
    t.long_name FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE
    m.season = '2008/2009'
AND t.long_name = 'KAS Eupen'
```

В ответе Metabase получим No results!, так как вместе с фильтром m.season = '2008/2009' исчезли все строки, значения которых NULL.

С LEFT JOIN также работают агрегатные функции, что позволяет не потерять значения из левой таблицы. Например, мы можем вывести сумму голов команд по гостевым матчам.

```sql
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM   
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY t.id
ORDER BY 2 DESC
```

__При применении функций SUM, MIN, MAX, AVG к полям со значением NULL в результате получится NULL, а не 0. А при использовании функции COUNT, наоборот, получится 0.__


При использовании RIGHT JOIN сохраняется та же логика, что и для LEFT JOIN, только за основу берётся правая таблица.

Чтобы из LEFT JOIN получить RIGHT JOIN, нужно просто поменять порядок соединения таблиц.

Вообще, применение RIGHT JOIN считается плохим тоном, так как язык SQL читается и пишется слева направо, а такой оператор усложняет чтение запросов.

### FULL OUTER JOIN
Оператор FULL OUTER JOIN объединяет в себе LEFT и RIGHT JOIN и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

FULL OUTER JOIN может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

_Пример: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа._

Синтаксис FULL OUTER JOIN аналогичен другим JOIN.

```sql
SELECT
…
FROM
	table1
FULL OUTER JOIN table2 ON условие
```


### CROSS JOIN

CROSS JOIN соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

Приведенные ниже три типа объединения дадут идентичный результат:

```sql
SELECT *
FROM
    sql.teams,
    sql.matches

--------------------------
SELECT *
FROM
    sql.teams
    CROSS JOIN sql.matches

--------------------------
SELECT *
FROM
    sql.teams
    JOIN sql.matches ON TRUE
```

__Обратите внимание! Условие для CROSS JOIN, в отличие от других операторов, не требуется.__

CROSS JOIN может быть полезен, когда необходимо создать таблицу фактов.

Например, с помощью такого запроса мы можем получить все возможные комбинации полных названий команд в матчах.

```sql
SELECT
    DISTINCT
    t1.long_name home_team,
    t2.long_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2
```


### NATURAL JOIN

Ключевое слово natural в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

NATURAL JOIN можно использовать с любыми видами соединений, которые требуют условия соединения:

1. NATURAL INNER JOIN (возможна запись NATURAL JOIN);
2. NATURAL LEFT JOIN;
3. NATURAL RIGHT JOIN;
4. NATURAL FULL OUTER JOIN.

__При использовании NATURAL JOIN прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц teams и matches этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец id, но таблицы соединяются по другим столбцам.__

Когда у таблиц есть несколько столбцов с одинаковыми именами, при NATURAL JOIN условие соединения будет применено на все столбцы с одинаковыми именами.

То есть для таблиц table1 и table2

table1: id, name, ...

table2: id, name, ...

будут эквиваленты запросы:

```sql
SELECT
    ...
FROM  
  table1 NATURAL JOIN table2

---------------------------------
SELECT
    …
FROM
  table1 t1 INNER JOIN table2 t2 ON t1.id = t2.id AND t1.name = t2.name
```

### ОБЩАЯ ЛОГИКА ПОСТРОЕНИЯ ЗАПРОСА С JOIN


__*При построении запроса с несколькими JOIN старайтесь идти слева направо. Сначала выберите таблицу, которая является центральной в соответствии с поставленной задачей, вопросом. Затем добавляйте таблицы поэтапно в зависимости от бизнес-логики запроса.*__

__чаще всего используются INNER JOIN и LEFT JOIN. Другие операторы используются реже, но стоит помнить об их существовании при решении нестандартных задач.__

## 6. Итоги. Закрепление знаний

__Если какую-то задачу не получается выполнить с первого раза — не отчаивайтесь. Соединение таблиц — одна из самых сложных тем для освоения и эффективного использования языка SQL.__

### ОТЛИЧНО, ВЫ СПРАВИЛИСЬ!

Однако мы предлагаем немного задержаться в этом модуле.

Объединение таблиц — одна из ключевых тем. Знания в этой области нередко проверяют на собеседованиях, поэтому уделите отработке навыков особое внимание. Ниже вы найдёте несколько наиболее распространённых вопросов по объединению таблиц.

*Все примеры взяты из книги Нолис Ж., Робинсон Э. Data Science для карьериста. СПб.: Питер, 2021.*

__Вопрос 1__
Задание от Лудамилы Джанда (Ludamila Janda),
Data Scientist в Amplify

Объясните разницу между левым (LEFT) и внутренним (INNER) соединениями.

ПРИМЕР ОТВЕТА
Соединения — это способы объединения данных из двух разных таблиц (левой и правой) в новую.

Принцип работы соединений заключается в объединении строк между двумя таблицами; набор ключевых столбцов используется для поиска данных в двух одинаковых таблицах, которые необходимо соединить.

В случае левого соединения каждая строка из левой таблицы появляется в полученной таблице, но строки из правой появляются, только если значения в их ключевых столбцах отображаются в левой. При внутреннем соединении обе строки из левой и правой таблиц появляются, только если есть совпадающая строка в другой таблице.

На практике вы можете воспринимать левое соединение как добавление данных из правой таблицы к левой, если она существует (например, с использованием правой таблицы в качестве подстановочной). Внутреннее соединение больше похоже на поиск всех общих данных и создание новой таблицы только из пар.


__Вопрос 2__

Перед вами школьная таблица TABLE_A с оценками от 0 до 100, выставленными ученикам нескольких классов.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/fbd472b425396a882520b25f8172106f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-u2-md3_6_1.png)


Как бы вы рассчитали максимальную оценку в каждом классе?

ПРИМЕР ОТВЕТА
Запрос для поиска наивысшей оценки в каждом классе может выглядеть так:

SELECT
  CLASS,
  MAX(GRADE) INTO TABLE_B
FROM
  TABLE_A
GROUP BY
  CLASS
Этот запрос группирует данные по каждому классу, а затем находит максимальное значение. Он дополнительно сохраняет результаты в новую таблицу (TABLE_B), чтобы их можно было запросить позже.


__Если вы услышали вопрос, который кажется вам слишком лёгким, вполне возможно, что таким он и является.__

Вопрос 3
Рассмотрим таблицу из предыдущего вопроса.

Как найти не только самую высокую оценку в каждом классе и учащегося, получившего её?

Предполагая, что у нас есть результат предыдущего запроса, сохранённый в TABLE_B, мы можем использовать его в этом решении:

SELECT
  a.CLASS,
  a.GRADE,
  a.STUDENT
FROM
  TABLE_A a
  INNER JOIN TABLE_B b ON a.CLASS = b.CLASS
  AND a.GRADE = b.GRADE
Этот запрос выбирает всех учащихся и их оценки из исходной таблицы TABLE_A, где указаны классы и оценки, которые отображаются в таблице максимальных значений TABLE_B. Внутреннее соединение работает как фильтр, сохраняя только максимальные комбинации классов/оценок, потому что только в этом случае оценка появляется в таблице TABLE_B.


В качестве альтернативы можно было бы использовать подзапрос, чтобы сделать то же самое, не вызывая таблицу TABLE_B:

SELECT
  a.CLASS,
  a.GRADE,
  a.STUDENT
FROM
  TABLE_A a
  INNER JOIN (
    SELECT
      CLASS,
      MAX(GRADE)
    FROM
      TABLE_A
    GROUP BY
      CLASS
  ) b ON a.CLASS = b.CLASS
  AND a.GRADE = b.GRADE


__Решение не даёт каких‐либо специфичных случаев для привязки к максимальному значению. В примере будут возвращены строки для нескольких студентов. Возможно, стоит указать на этот факт интервьюеру, потому что так вы показываете, что обращаете внимание на пограничные случаи.__